In [112]:
%matplotlib inline
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import pandas as pd
import os
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pickle

In [113]:
rent = "final.csv"
rent_df = pd.read_csv(rent)
rent_df = rent_df.drop(["Unnamed: 0"], axis = 1)
#rent_df.dropna(inplace = True)

In [114]:
indexNames = rent_df[rent_df["Rent"] == rent_df ["chrent"]].index
rent_df.drop(indexNames, inplace = True)
outliers = rent_df[rent_df["chrent"] >= 500].index
rent_df.drop(outliers, inplace = True)
outliers2 = rent_df[rent_df["chrent"] <= -500].index
rent_df.drop(outliers2, inplace = True)
rent_df.drop(["LengthStay","DateOccurred","DateMoveOut"], axis = 1, inplace =True)
rent_df.dropna(inplace = True)
rent_df

,Property_Code,Tenant_Code,Event,Unit_Code,DateMoveIn,Rent,Vacancy,SQFT,BedRooms,Bathroom,chrent
1,ce,t0013745,Rent Change,C05266,2006-11-09,896.0,1,1400,3,2.5,26.0
4,ce,t0013748,Rent Change,C05272,2012-09-28,800.0,0,1088,2,1.5,30.0
5,ce,t0013748,Rent Change,C05272,2012-09-28,824.0,1,1088,2,1.5,24.0
9,ce,t0013749,Rent Change,C05274,2008-11-26,915.0,0,1400,3,2.5,35.0
10,ce,t0013749,Rent Change,C05274,2008-11-26,924.0,0,1400,3,2.5,9.0
11,ce,t0013749,Rent Change,C05274,2008-11-26,970.0,0,1400,3,2.5,46.0
12,ce,t0013749,Rent Change,C05274,2008-11-26,1044.0,0,1400,3,2.5,74.0
14,ce,t0013750,Rent Change,C05276,2001-07-21,890.0,0,1400,3,2.5,10.0
15,ce,t0013750,Rent Change,C05276,2001-07-21,917.0,0,1400,3,2.5,27.0
16,ce,t0013750,Rent Change,C05276,2001-07-21,945.0,0,1400,3,2.5,28.0


In [115]:
X = rent_df[["SQFT", "BedRooms", "Bathroom", "chrent", "Property_Code"]]
y = rent_df["Vacancy"]
print(X.shape, y.shape)

(6096, 5) (6096,)


In [116]:
data = X.copy()

data_binary_encoded = pd.get_dummies(data, columns = ["Property_Code"])
data_binary_encoded.head()

,SQFT,BedRooms,Bathroom,chrent,Property_Code_ce,Property_Code_fh,Property_Code_mpg,Property_Code_sh
1,1400,3,2.5,26.0,1,0,0,0
4,1088,2,1.5,30.0,1,0,0,0
5,1088,2,1.5,24.0,1,0,0,0
9,1400,3,2.5,35.0,1,0,0,0
10,1400,3,2.5,9.0,1,0,0,0


In [117]:
#data_binary_encoded.dropna(inplace = True)

In [118]:
data_binary_encoded.dtypes

SQFT                   int64
BedRooms               int64
Bathroom             float64
chrent               float64
Property_Code_ce       uint8
Property_Code_fh       uint8
Property_Code_mpg      uint8
Property_Code_sh       uint8
dtype: object

In [119]:
data_binary_encoded ["BedRooms"]= data_binary_encoded["BedRooms"].astype(float)
data_binary_encoded ["Property_Code_ce"]= data_binary_encoded["Property_Code_ce"].astype(float)
data_binary_encoded ["Property_Code_fh"]= data_binary_encoded["Property_Code_fh"].astype(float)
data_binary_encoded ["Property_Code_mpg"]= data_binary_encoded["Property_Code_mpg"].astype(float)
data_binary_encoded ["Property_Code_sh"]= data_binary_encoded["Property_Code_sh"].astype(float)

In [120]:
print (data_binary_encoded.shape)
print (y.shape)

(6096, 8)
(6096,)


In [121]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(data_binary_encoded, y, stratify=y, random_state=42)

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [122]:
print(X_test.shape)
print (y_test_categorical.shape)
print (X_train.shape)

(1524, 8)
(1524, 2)
(4572, 8)


In [123]:
from keras.models import Sequential
from keras.layers import Dense
deep_model = Sequential()

deep_model.add(Dense(units=100, activation='relu', input_dim=8))
deep_model.add(Dense(units=100, activation='relu'))
deep_model.add(Dense(units=2, activation='softmax'))

In [131]:
deep_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

deep_model.fit(
    X_train,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
 - 1s - loss: 4.0789 - acc: 0.7469
Epoch 2/100
 - 0s - loss: 4.0789 - acc: 0.7469
Epoch 3/100
 - 0s - loss: 4.0789 - acc: 0.7469
Epoch 4/100
 - 0s - loss: 4.0789 - acc: 0.7469
Epoch 5/100
 - 0s - loss: 4.0789 - acc: 0.7469
Epoch 6/100
 - 0s - loss: 4.0789 - acc: 0.7469
Epoch 7/100
 - 0s - loss: 4.0789 - acc: 0.7469
Epoch 8/100
 - 0s - loss: 4.0789 - acc: 0.7469
Epoch 9/100
 - 0s - loss: 4.0789 - acc: 0.7469
Epoch 10/100
 - 0s - loss: 4.0789 - acc: 0.7469
Epoch 11/100
 - 0s - loss: 4.0789 - acc: 0.7469
Epoch 12/100
 - 0s - loss: 4.0789 - acc: 0.7469
Epoch 13/100
 - 0s - loss: 4.0789 - acc: 0.7469
Epoch 14/100
 - 0s - loss: 4.0789 - acc: 0.7469
Epoch 15/100
 - 0s - loss: 4.0789 - acc: 0.7469
Epoch 16/100
 - 0s - loss: 4.0789 - acc: 0.7469
Epoch 17/100
 - 0s - loss: 4.0789 - acc: 0.7469
Epoch 18/100
 - 0s - loss: 4.0789 - acc: 0.7469
Epoch 19/100
 - 0s - loss: 4.0789 - acc: 0.7469
Epoch 20/100
 - 0s - loss: 4.0789 - acc: 0.7469
Epoch 21/100
 - 0s - loss: 4.0789 - acc: 0.7469
E

In [125]:
model_loss, model_accuracy = deep_model.evaluate(
    X_test, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Deep Neural Network - Loss: 4.0718285918548665, Accuracy: 0.7473753280839895


In [126]:
deep_predictions = deep_model.predict_classes(X_train)
deep_predictor = pd.DataFrame({"Predictions": deep_predictions, "Actual": y_train})
deep_predictor.loc[deep_predictor["Predictions"] == 1, "Predictions"] = "Vacant"
deep_predictor.loc[deep_predictor["Predictions"] == 0, "Predictions"] = "Occupied"
deep_predictor.loc[deep_predictor["Actual"] == 1, "Actual"] = "Vacant"
deep_predictor.loc[deep_predictor["Actual"] == 0, "Actual"] = "Occupied"
deep_predictor


,Predictions,Actual
4350,Occupied,Occupied
3479,Occupied,Vacant
5330,Occupied,Vacant
1952,Occupied,Occupied
2259,Occupied,Vacant
4662,Occupied,Vacant
4878,Occupied,Vacant
6107,Occupied,Occupied
4391,Occupied,Occupied
4130,Occupied,Occupied


In [127]:
deep_model.save("logistics_deep_network.h5")

In [128]:
filename = "logisitic_depp_network.pkl"
list_pickle = open(filename, "wb")
pickle.dump(deep_model, list_pickle)
list_pickle.close()

In [129]:
deep_model.predict_classes([[[1400,3,2.5,26.0,1,0,0,0]]])

array([0], dtype=int64)